# Firework Algorithm for Optimization

- Autores originais: [*Ying Tan*](https://scholar.google.com/citations?user=PjNxSPsAAAAJ&hl=en) e [*Yuanchun Zhu*](https://ieeexplore.ieee.org/author/38008573100)
- Ano de Publicação: 2010
- Editora: [Springer](https://link.springer.com/chapter/10.1007/978-3-642-13495-1_44)
- Conferência: [Advances in Swarm Intelligence](https://link.springer.com/book/10.1007/978-3-642-13495-1): First International Conference, ICSI 2010, Beijing, China, June 12-15, 2010, Proceedings, Part I 1

## Descrição:
FA é um algoritmo de inteligência de enxames inspirado na explosão de fogos de artifício. Esse algoritmo é utilizado para a otimização global de funções complexas. São fornecidos dois processos de *“explosão”* (busca) e os mecanismos para manter a diversidade das *“faíscas”*.

In [10]:
import numpy as np
import pandas as pd
import random

## Funções Objetivos

In [11]:
def sphere_function(x):
    result = x**2
    return np.sum(result)

In [12]:
def rastrigin(x):
    n = len(x)
    total = 0
    total = x**2 - 10*np.cos(2*np.pi*x)
    return (10*n) + total

In [13]:
def rosenbrook(x):
    n = len(x)
    term1 = 100 * (x[1:] - x[:-1]**2)**2
    temr2 = (x[1:] -1)**2
    sumterm = np.sum(term1 + term2)
    return sumterm

In [14]:
def ackley(x):
    n = len(x)
    sum1 = np.sum(x*2)
    sum2 = np.sum(np.cos(2 * np.pi * x**2))
    term1 = -20 * np.exp(-0.2 * np.sqrt(1/n * sum1))
    term2 = -np.exp(1/n * sum2)
    return term1 + term2 + 20 + np.exp(1)

### Distribuição Gaussiana:

In [15]:
value = random.gauss(1,1)
value

0.8875296029796005

### Distribuição Uniforme:

In [16]:
value = random.uniform(-1,1)
value

-0.6262641041074666

## Inicializando posições dos Fogos de Artifício:

In [17]:
def initialize_firework_locations(n, d, xmin, xmax):
    fireworks = xmin + (xmax - xmin) * np.random.rand(n,d)
    return fireworks

## Calculando o número de faíscas:

In [18]:
def sparks_number(fx, fX, m, ymax, eps=.00001):
    sum_difference_of_worst_fit_and_fx = np.sum(ymax-fX) + eps
    difference_of_max_and_fx = (ymax - fx + eps)
    s = m * difference_of_max_and_fx/sum_difference_of_worst_fit_and_fx
    s = firework_boundaries(0.1, 0.8, m, s)
    return s

In [19]:
def firework_boundaries(a, b, m, s):
    term1 = np.round(a*m) * (s < (a*m))
    term2 = np.round(b*m) * (s> b*m)
    term3 = np.round(s) * (~(s < (a*m)) & ~((s > b *m) & (a<b<1)))
    s_new = term1 + term2 + term3
    return s_new

## Obtendo localização das Faíscas:

### Seletor de dimensões aleatório

In [20]:
def random_dimension_selector(d):
    xj = np.zeros(d)
    z = round(d * random.uniform(0,1))
    random_dimensions = np.random.choice(d, z, replace=False)
    for i in random_dimensions:
        xj[i] = 1
    return xj
    

### Amplitude de explosão

In [21]:
def explosion_amplitude(A_hat, fx,fX, ymin, eps=.00001):
    sum_difference_of_fx_and_best_fit = np.sum(fX - ymin) + eps
    difference_of_fx_and_min = (fx - ymin + eps)
    A = A_hat * difference_of_fx_and_min/sum_difference_of_fx_and_best_fit
    return A

In [22]:
d = 6
round(6 * random.uniform(0,1))

1

### Obtendo faísca: Estratégia da Explosão

In [23]:
def obtain_location_of_spark(d, amp, xmin, xmax, xi):
    #selecting z dimensions to be affected
    xj = xi
    random_selected_positions = random_dimension_selector(d) 
    h = amp * random.uniform(-1 ,1) 
    random_affected_dimensions = xj + (random_selected_positions* h)
    #mapping affected dimensions to potential space
    random_affected_dimensions = map_to_potential_space(xmin, xmax, random_affected_dimensions)
    return random_affected_dimensions

### Mapeamento para espaço potencial:

In [24]:
def map_to_potential_space(xmin, xmax, affected):
    inbound = xmin + abs(affected)%(xmax - xmin)
    term1 = (affected > xmax) * inbound
    term2 =  (affected < xmin) * inbound
    fixed_bounds = term1 + term2
    mapping = (fixed_bounds == 0) * affected + fixed_bounds 
    return mapping

In [25]:
xi = np.random.rand(3)
xmin = np.array([-3.12, -4.5, -6.6])
xmax = np.array([3.12, 4.5, 6.6])
d = 3
amp = 5
obtain_location_of_spark(d, amp, xmin, xmax, xi)

array([ 0.51426892, -3.32213318,  0.53298253])

### Obtendo faísca: Estratégia de Faíscas Gaussianas:

In [26]:
def gaussian_spark_location(xmin, xmax, d, xi):
    xj = xi
    random_dimensions_selected = random_dimension_selector(d)
    random_affected_dimensions = (xj * np.logical_not(random_dimensions_selected)) + xj * (random_dimensions_selected * np.random.normal(1,1,d))
    random_affected_dimensions = map_to_potential_space(xmin, xmax, random_affected_dimensions)
    return random_affected_dimensions

In [27]:
d =3
np.random.normal(1,1,d)

array([ 2.00417187,  2.13230407, -0.62905488])

In [28]:
np.random.rand(3)

array([0.54020565, 0.12537599, 0.14140263])

## Seleção de Localizações:

### Calculando Distância Geral:

In [29]:
def general_distance(xi , X, d):
    sum_of_distances = np.sum(abs(xi - X))
    return sum_of_distances

### Calculando probabilidade de seleção:

In [30]:
def location_probability(RX, Rxi,d):
    pxi = Rxi/RX
    return pxi

## Framework ALgoritmo de Otimização Fogos de Artifício:

In [120]:
#localizações
n = 10
#quantidade de faíscas
m = 50
#dimensões
d = 10
#parâmetro para amplitude
a = 20
#faíscas gaussianas
gauss = 15

#Número de fogos na estratégia gaussiana
m_hat = 5

#minimo valor do espaço potencial
xmin = -5

#máximo valor do espaço potencial
xmax = 5

#Inicializando fogoso de artifício
X = initialize_firework_locations(n, d, xmin, xmax)

#Total de iterações
explosion_generations = 10

current_generation = 0

    #f(X)
y = np.zeros((n,1))
for i in range(n):
    y[i] = sphere_function(X[i])

#best fit
ymin = y.min()
#worst fit
ymax = y.max()

#Numero de faíscas por fogo de artifício
S = np.zeros((n,1))
#Amplitude de cada fogo de artificio
A = np.zeros((n,1))

for i in range(n):
    S[i] = sparks_number(y[i],y, m, ymax) 
    A[i] = explosion_amplitude(a, y[i],y, ymin)

total_sparks = np.sum(S)
new_locations = X
generation = np.array((n,d))
for i in range(n):
    xi = X[i]
    n_sparks = int(S[i][0])
    sparks = np.zeros((n_sparks,d))
    for j in range(n_sparks):
        sparks[j] = obtain_location_of_spark(d, a, xmin, xmax, xi)
    new_locations = np.vstack((new_locations, sparks)) 

random_fireworks = np.sort(np.random.choice(n,m_hat, replace=False))
for i in range(m_hat):
    xi = X[random_fireworks[i]]
    gaussian_sparks = np.zeros((gauss,d))
    for j in range(gauss):
        gaussian_sparks[j] = gaussian_spark_location(xmin, xmax, d, xi)
    new_locations = np.vstack((new_locations, gaussian_sparks)) 
total_locations = int(total_sparks) + n + m_hat*gauss
fx_new_locations = np.zeros((total_locations, 1))
for i in range(total_locations):
    fx_new_locations[i] = sphere_function(new_locations[i])

current_best_location = fx_new_locations.argmin()
best_location = new_locations[current_best_location]
new_locations = np.delete(new_locations, current_best_location,0)
RX = np.zeros((total_locations -1, 1)) 
for i in range(total_locations-1):
    RX[i] = general_distance(new_locations[i], new_locations, d)

prob = np.zeros((total_locations, 1))
sum_RX = np.sum(RX)
for i in range(total_locations-1):
    prob[i] = location_probability(sum_RX, RX[i],d)
prob = prob.flatten()
indexes = np.arange(total_locations)

next_generation_selection = np.random.choice(indexes, n -1, p=prob, replace=False)

X = np.zeros((n-1,d))
for i in range(n-1):
    X[i] = next_generation_selection[i]
X = np.vstack((best_location, X))
print("current best location:\n",best_location)


current best location:
 [-0.81644327 -0.91303287  0.70230129 -2.3737117   2.41173736 -0.54810815
  0.76710885 -0.31725263 -2.1621021   0.00368007]


In [32]:
fx = y[4]
si = sparks_number(fx, y, m, ymax)
print("si: ",si)
print("fx: ", fx)
print("ymax: ",ymax)
sum = np.sum(ymax - y)
print("sum: ",sum)
spark =  (ymax - fx)/sum
print("ymax - y[0]/sum: ", spark)
print("m * ymax - fx/sum: ",m* spark)

si:  [5.]
fx:  [93.62072488]
ymax:  114.62867243142851
sum:  326.9434401436402
ymax - y[0]/sum:  [0.0642556]
m * ymax - fx/sum:  [3.2127801]


In [33]:
total_fireworks = np.sum(S)
total_fireworks

62.0

In [34]:
np.sort(np.random.choice(10,5, replace=False))

array([1, 3, 4, 6, 9])

In [35]:
xi = X[0]
gaussian_spark_location(xmin, xmax, d, xi)

array([-0.5478428 , -2.49553569, -1.92993621, -0.19739772,  3.61693716,
        1.18483508, -4.01740075,  3.27813369,  2.05261901,  0.37253318])

In [36]:
new_locations[101]

array([ 3.63933362,  3.09253723, -3.43780373,  4.87334744, -4.98926002,
       -0.13569889,  4.67454462, -3.85276055,  2.01691299, -1.35569374])

In [37]:
new_locations[69:]

array([[ 3.74931957, -3.3901132 ,  1.00118681,  0.37352948,  2.27275181,
         2.75840581, -4.75793008, -1.92461376, -0.58346312,  3.60840001],
       [ 3.11565189,  0.25508467,  1.00118681,  0.37352948,  2.27275181,
         2.75840581,  1.62290154,  3.44838199, -0.58346312,  3.98139576],
       [ 0.67953011, -2.83756922, -0.01098129,  1.39157893,  1.25770793,
         1.12968277, -2.62778941,  1.49368961,  1.60556048, -4.26494126],
       [ 1.19488985, -1.69916705,  0.05020178,  1.39157893,  1.25770793,
         1.12968277,  1.42872907, -4.99807856,  1.60556048, -4.44194145],
       [ 1.14163011, -2.83756922,  0.05020178,  1.39157893,  3.53856236,
         1.12968277,  4.74035321, -4.99807856,  1.60556048, -4.44194145],
       [ 1.14163011, -4.32924973,  0.05881962,  0.32587392,  2.1696192 ,
         1.12968277, -1.22136631, -2.21675928, -0.94576127, -4.47140739],
       [ 0.21075148,  2.71972453,  0.04878913,  3.90705611,  1.91998087,
         2.77552772,  3.22218671, -0.86427325

In [38]:
current_best_location

70

In [71]:
next_generation_selection[0]

IndexError: invalid index to scalar variable.

In [97]:
print(indexes.shape)
print(prob)
next_gen = np.random.choice(indexes, total_locations -1, p=prob, replace=False)
print("NEXT GEN SIZE: ", len(next_gen))
print("total locations:", total_locations)

(146,)
[0.00867243 0.00703491 0.0067689  0.00628979 0.00690984 0.00601012
 0.00772303 0.00768895 0.00624476 0.00636884 0.00812837 0.00950169
 0.00767612 0.00705985 0.00794581 0.00711201 0.00713    0.00732064
 0.00727127 0.0067499  0.00641714 0.00715639 0.0081243  0.0079248
 0.00865838 0.00648492 0.0060099  0.00663166 0.00645732 0.00580073
 0.00739646 0.00673709 0.00691657 0.00793621 0.00599925 0.00686246
 0.00691302 0.00653242 0.00616618 0.00604553 0.00685695 0.00631654
 0.00623073 0.00669889 0.00739735 0.00772303 0.00679738 0.00772303
 0.0070962  0.00769722 0.00783018 0.00746205 0.006026   0.00721861
 0.00574362 0.006673   0.00624476 0.00617448 0.00683413 0.00601342
 0.00735298 0.00596702 0.00782632 0.00768042 0.00708799 0.00679616
 0.00766761 0.00633692 0.00771842 0.00704899 0.00787659 0.00729126
 0.00694156 0.0068163  0.00810337 0.00772671 0.0081495  0.00816535
 0.00771402 0.00841735 0.00792841 0.00886334 0.00739703 0.00819531
 0.00849057 0.0062153  0.00650235 0.00698255 0.00642763 